In [9]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from collections import defaultdict
import os
import sys

sys.path.append(os.path.abspath(os.pardir))

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

from tdparse.helper import read_config, full_path
from tdparse.parsers import semeval_14, semeval_15_16, dong, election
from tdparse.data_types import TargetCollection
from tdparse.tokenisers import ark_twokenize

In [2]:
# Load all of the datasets
semeval_14_rest_train = semeval_14(full_path(read_config('semeval_2014_rest_train')))
semeval_14_lap_train = semeval_14(full_path(read_config('semeval_2014_lap_train')))
semeval_14_rest_test = semeval_14(full_path(read_config('semeval_2014_rest_test')))
semeval_14_lap_test = semeval_14(full_path(read_config('semeval_2014_lap_test')))
semeval_15_rest_test = semeval_15_16(full_path(read_config('semeval_2015_rest_test')))
semeval_16_rest_test = semeval_15_16(full_path(read_config('semeval_2016_rest_test')),
                                     sep_16_from_15=True)
dong_train = dong(full_path(read_config('dong_twit_train_data')))
dong_test = dong(full_path(read_config('dong_twit_test_data')))
election_train, election_test = election(full_path(read_config('election_folder_dir')))
# Product reviews are made up of three different products: 1. Computer, 2. Router, and 3. Speaker
product_reviews_folder = full_path(read_config('product_reviews_dir'))
speaker_reviews = semeval_14(os.path.join(product_reviews_folder, 'Speaker.xml'))
computer_reviews = semeval_14(os.path.join(product_reviews_folder, 'Computer.xml'))
router_reviews = semeval_14(os.path.join(product_reviews_folder, 'Router.xml'))

youtubean = semeval_14(full_path(read_config('youtubean')))
semeval_14_rest = TargetCollection.combine_collections(semeval_14_rest_train,
                                                           semeval_14_rest_test)
semeval_14_laptop = TargetCollection.combine_collections(semeval_14_lap_train,
                                                         semeval_14_lap_test)
semeval_15_rest = semeval_15_16(full_path(read_config('semeval_2015_rest_test')))
semeval_16_rest = semeval_15_16(full_path(read_config('semeval_2016_rest_test')),
                                     sep_16_from_15=True)
dong = TargetCollection.combine_collections(dong_train, dong_test)
election = TargetCollection.combine_collections(election_train, election_test)
# Combine all of the product reviews
product_reviews = TargetCollection.combine_collections(speaker_reviews, computer_reviews,
                                                       router_reviews)
datasets = {'SemEval 14 Laptop' : semeval_14_laptop, 'SemEval 14 Resturant' : semeval_14_rest,
            'SemEval 15 Resturant' : semeval_15_rest, 'SemEval 16 Resturant' : semeval_16_rest,
            'Product Reviews' : product_reviews, 'Dong Twitter' : dong, 
            'Election Twitter' : election, 'YouTuBean' : youtubean}

# Datasets
This notebook will describe the different datasets that have been used as well as the statistics of these datasets. The datasets used are the following:
1. [Dong et al.](https://aclanthology.coli.uni-saarland.de/papers/P14-2009/p14-2009) [Twitter dataset](https://github.com/bluemonk482/tdparse/tree/master/data/lidong) NOTE that the dataset does not link to the paper as the dataset released from the paper has already been pre-processed where as this dataset has not.
2. [SemEval 2014 Resturant dataset](http://alt.qcri.org/semeval2014/task4/index.php?id=data-and-tools). We used Train dataset version 2 and the test dataset. This dataset contains 4 sentiment values; 1. Positive, 2. Neutral, 3. Negative, and 4. Conflict but we are only going to use the first 3 to make it comparable to the other datasets and the fact that the conflict label only has 91 instances in the training set and 14 in the test set.
3. [SemEval 2014 Laptop dataset](http://alt.qcri.org/semeval2014/task4/index.php?id=data-and-tools). We used Train dataset version2 and the test dataset. This dataset contains 4 sentiment values; 1. Positive, 2. Neutral, 3. Negative, and 4. Conflict but we are only going to use the first 3 to make it comparable to the other datasets and the fact that the conflict label only has 45 instances in the training set and 16 in the test set.
4. [SemEval 2015 Resturant dataset](). We only used the test dataset as the training set has overlap with the original 2014 version.
5. [SemEval 2016 Resturant dataset](http://alt.qcri.org/semeval2016/task5/index.php?id=data-and-tools). We only used the test dataset as the training set has overlap with the original 2014 version.
6. [Election dataset](https://figshare.com/articles/EACL_2017_-_Multi-target_UK_election_Twitter_sentiment_corpus/4479563/1)
7. [Product review by Liu et al, IJCAI-2015](https://www.aaai.org/ocs/index.php/IJCAI/IJCAI15/paper/view/10766) [dataset](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#datasets)
8. [Youtubean dataset](https://github.com/epochx/opinatt/blob/master/samsung_galaxy_s5.xml) [by Marrese-Taylor et al.](https://www.aclweb.org/anthology/W17-5213) - Dataset of 7 youtube reviews of the Samsung Galaxy S5. The text are the closed captions of the videos where the captions were provided by the authors and not automatically generated.

In [3]:
dataset_dict = defaultdict(list)
index = []
columns = ['No. Targets (Dataset Size)', 'No. Senti Labels', 
          'Mean Targets per Sent', 'No Unique Targets',
          '% Targets with 1 Sentiment per Sentence', '% Targets with 2 Sentiment per Sentence', 
          '% Targets with 3 Sentiment per Sentence', 'Avg sentence length per target']
for name, dataset in datasets.items():
    index.append(name)
    targets_i_senti = []
    num_targets = len(dataset)
    num_sentiment_labels = len(dataset.stored_sentiments())
    avg_sent_length = dataset.avg_sentence_length_per_target()
    for i in range(1, 4):
        if i > num_sentiment_labels:
            targets_i_senti.append(0)
        else:
            i_senti_targets = len(dataset.subset_by_sentiment(i))
            targets_i_senti\
            .append((i_senti_targets / num_targets) * 100)
    
    dataset_dict['No. Targets (Dataset Size)'].append(num_targets)
    dataset_dict['No. Senti Labels'].append(num_sentiment_labels)
    dataset_dict['Mean Targets per Sent'].append(dataset\
                                                 .avg_targets_per_sentence())
    dataset_dict['No Unique Targets'].append(dataset.number_unique_targets())
    dataset_dict['% Targets with 1 Sentiment per Sentence'].append(targets_i_senti[0])
    dataset_dict['% Targets with 2 Sentiment per Sentence'].append(targets_i_senti[1])
    dataset_dict['% Targets with 3 Sentiment per Sentence'].append(targets_i_senti[2])
    dataset_dict['Avg sentence length per target'].append(avg_sent_length, ark_twokenize)
    

dataset_stats = pd.DataFrame(dataset_dict, index=index, columns=columns)
dataset_stats.round(2)

,No. Targets (Dataset Size),No. Senti Labels,Mean Targets per Sent,No Unique Targets,% Targets with 1 Sentiment per Sentence,% Targets with 2 Sentiment per Sentence,% Targets with 3 Sentiment per Sentence,Avg sentence length per target
SemEval 14 Laptop,2951,3,1.58,1295,81.09,17.62,1.29,18.57
SemEval 14 Resturant,4722,3,1.83,1630,75.26,22.94,1.80,17.25
SemEval 15 Resturant,597,3,1.49,269,79.90,19.60,0.50,15.85
SemEval 16 Resturant,649,3,1.55,312,89.83,9.71,0.46,16.36
Product Reviews,1101,2,1.40,468,94.37,5.63,0.00,20.12
Dong Twitter,6940,3,1.00,145,100.00,0.00,0.00,17.37
Election Twitter,11899,3,2.94,2190,44.50,46.72,8.78,21.68
YouTuBean,798,3,2.07,522,81.45,18.17,0.38,22.53


The high level statistics are presented above. At first it is a surprising that the Twittwer data has a higher average sentence length but the sentence in the Twitter cases is actually a Tweet compared to the product, SemEval, and YouTuBean data which has been sentence split. However the YouTuBean data even when sentence split is still the longest this could be due to the data being speech text rather than written.

Again the datasets vary with the number of Targets with distinct sentiments per sentence but most have only one distinct sentiment per sentence apart from the Election dataset which has quiet an even split between 1 and 2 distinct sentiments.

Lastly the Election dataset has the highest number of targets per sentence by a long way and this is not porportinal to the average sentence length either.

## Syntactic Complexity of the dataset
The above statistics are all based on quiet high level summary statistics and do not contain any lingustic specfic statistic apart from perhaps the average sentence length. Therefore below is the table of average constituency tree depth for the datasets which can be viewed as showing the sentence syntax complexity this was also shown in the [Marrese-Taylor et al.](https://www.aclweb.org/anthology/W17-5213) paper on the datasets they used and here we present the same statistic on a large number of datasets.

In [12]:
dataset_ling_dict = defaultdict(list)
index = []
columns = ['Average constituency tree depth']
for name, dataset in datasets.items():
    index.append(name)
    dataset_ling_dict['Average constituency tree depth'].append(dataset.avg_constituency_depth())

In [18]:
dataset_ling_stats = pd.DataFrame(dataset_ling_dict, index=index, columns=columns)
dataset_ling_stats.round(2)

,Average constituency tree depth
SemEval 14 Laptop,10.61
SemEval 14 Resturant,9.73
SemEval 15 Resturant,9.51
SemEval 16 Resturant,9.00
Product Reviews,9.95
Dong Twitter,8.34
Election Twitter,9.67
YouTuBean,11.72


As you can see above interestingly the Election dataset that had the 2nd largest average sentence length and by far the largest number of targets per sentence does not have the largest average tree depth. The YouTuBean dataset does which may suggest that spoken text is syntactically more complex then written text. 